# 图书推荐教程
这个笔记本展示了如何对原始数据集进行预处理、特征提取、模型训练、评估等，最终将数据传送至远端服务器上进行推荐。笔记本中可能需要修改的是原始数据的存放路径，即第4个单元格中的DataDir

In [1]:
from google.colab import drive    # 先挂载存放有原始数据的云端硬盘
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!git clone https://github.com/Mikeaser/BookRecSystem.git  # 这个仓库是本次参赛的项目
!pip install annoy # colab中只要安装一个包

fatal: destination path 'BookRecSystem' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
ProjectFolderDir = '/content/BookRecSystem/'   # 项目文件夹的路径是相对于本笔记本路径而言的，当然也可以使用绝对路径
DataDir = '/content/drive/MyDrive/DataSpace/' # 原始数据路径，根据实际情况修改

初步分析阶段先将字段以字符串形式录入，便于后期统一格式

In [4]:
BigData = pd.read_csv(DataDir + "浙江大学-图书外借数据-2013~2018.csv", encoding='utf8', dtype=str)      

In [5]:
SmallData = pd.read_csv(DataDir + "浙江大学-图书外借数据-2019.csv", encoding='utf8', dtype=str)         

下面是构建模型所需要的数据列

In [6]:
user_columns = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_columns = ['ITEM_ID', 'SUBLIBRARY', 'ITEM_CALLNO', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         图书索书号        出版年          作者       题目    出版社
time = 'LOAN_DATE'

将两份数据集合并

In [7]:
UsedColumns = user_columns + item_columns + [time]
UsedData = pd.concat([BigData[UsedColumns], SmallData[UsedColumns]])

本项目将借阅记录中的空值字段采用相同值’na'进行填充，特征处理时将其作为空特征。对于图书类别，将其索书号按/分割后得到图书大类CALLNO1与小类CALLNO2，将借阅日期转为整形，便于之后比较大小。

In [8]:
import re

def get_Bletter(str0):   # 取出大写字母
    b = re.sub(u"([^\u0041-\u007a])", "", str0)
    return b

UsedData = UsedData.fillna(value='na')
UsedData['CALLNO1'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[0].map(lambda x: get_Bletter(str(x)))
UsedData['CALLNO2'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[1].map(lambda x: get_Bletter(str(x)))
UsedData[time]=UsedData[time].astype(int)
UsedData = UsedData.drop(columns='ITEM_CALLNO')  # 删掉不再需要的列

此时可以看到UsedData中的空值消失，并且多出了两个类别列

In [9]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


将数据存入./data/processed中，作为后续分析数据

In [10]:
import os


isExists=os.path.exists(ProjectFolderDir+"data/processed/")
if not isExists:
    os.makedirs(ProjectFolderDir+"data/processed/") 

UsedData.to_csv(ProjectFolderDir+"data/processed/ZJULibrary2013_2019.csv")

In [11]:
del BigData # 将之前的大数据变量清空，空出一部分内存
del SmallData
del UsedData

调用代码库中的主函数进行模型训练、评估等操作

In [12]:
import sys
sys.path.append(ProjectFolderDir)

from scripts.MainFile import main
from utils.data import datapretreat, MatchDataGenerator
from utils.features import SequenceFeature, SparseFeature
from model.DSSM import DSSM
import torch
from utils.train import MatchTrainer
import numpy as np
import random
from utils.recall import topn_evaluate

# 固定随机数种子
seed = 2023
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# 创建缓存目录

isExist=os.path.exists(ProjectFolderDir+"temp/")
if not isExist:
    os.makedirs(ProjectFolderDir+"temp/")


调用主函数即可，运行的日志文件保存在ProjectFolderDir/log目录下

In [13]:
main(ProjectFolderDir,  # 之前定义的项目路径
      neg_ratio= 3,  # 负采样倍率
      min_item= 5,  # 最短序列要求
      seq_max_len= 10,  # 截断序列长度
      load= False , # 是否从已有数据读取
      batch_size= 2048 , # batch大小
      user_params= [512, 512, 256, 128, 64] , # 读者塔MLP参数
      item_params= [1024, 512, 256, 128, 64] , # 图书塔MLP参数，要确保最后的维度一致
      temperature= 0.02 , # 温度系数
      learning_rate= 0.01 , # 学习率
      weight_decay= 1e-4 , # 正则化系数
      optimizer_fn= torch.optim.Adam ,  # 优化器
      epoch= 10 , # 训练epoch
      topk= 100 #推荐topk个商品 
    )

Start: 2023-03-09 07:18:01


loading data:: 100%|██████████| 8/8 [00:00<00:00, 54471.48it/s]


standard data has been generated
epoch: 0


train:  16%|█▌        | 641/4067 [01:33<08:19,  6.85it/s, loss=0.513]


KeyboardInterrupt: ignored

运行结束后，会产生文件

{ProjectFolderDir}/data/processed/ZJULibrary2013_2019.csv # 合并整理后的数据集

{ProjectFolderDir}/data/processed/data_process.npy # 训练集，验证集，测试集

{ProjectFolderDir}/data/processed/raw_id_maps.npy # 原始的id字典  

{ProjectFolderDir}/log/{Start time}.txt # 以开始训练时间命名的日志文件

{ProjectFolderDir}/temp/item.ann.index # 保存的用于召回的索引文件

{ProjectFolderDir}/temp/item_embeddding.pth # 图书的embedding向量

{ProjectFolderDir}/temp/user_embedding.pth # 读者的embedding向量

{ProjectFolderDir}/temp/model.pth # 训练的模型参数